# [Database Secret Engine](https://developer.hashicorp.com/vault/tutorials/db-credentials/database-secrets)

In [1]:
%env  WORKDIR=/tmp/vault                 

env: WORKDIR=/tmp/vault


In [2]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.IjWOxR9d1SAPnTJY1IidfVNE


In [3]:
# Replace VAULT_TOKEN with output value from previous cell
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.IjWOxR9d1SAPnTJY1IidfVNE
%env VAULT_CACERT=/tmp/vault/vault.ca

env: VAULT_ADDR=https://127.0.0.1:8200
env: VAULT_TOKEN=hvs.IjWOxR9d1SAPnTJY1IidfVNE
env: VAULT_CACERT=/tmp/vault/vault.ca


## Step 1: Install PostgreSQL

In [6]:
%%bash
# Add Helm repository by Bitnami
helm repo add bitnami https://charts.bitnami.com/bitnami

# Update Helm index charts
helm repo update
kubectl apply -f local-pv.yaml
kubectl apply -f pv-claim.yaml
# volumePermissions for EKS
helm install postgresql-dev -f postgres.yaml bitnami/postgresql -n vault --set volumePermissions.enabled=true

"bitnami" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "jaeger-all-in-one" chart repository
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "jaegertracing" chart repository
...Successfully got an update from the "signoz" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "external-secrets" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "bitnami" chart repository
...Successfully got an update from the "gitlab" chart repository
Update Complete. ⎈Happy Helming!⎈
persistentvolume/postgresql-data unchanged
persistentvolumeclaim/postgresql-data-claim unchanged
NAME: postgresql-dev
LAST DEPLOYED:

## Enable the database secret engine

In [7]:
%%bash
export POSTGRES_URL=postgresql-dev.vault.svc.cluster.local
# Enable engine
vault secrets enable database

# Configure the database secrets engine with the connection credentials for the Postgres database.
vault write database/config/postgresql \
     plugin_name=postgresql-database-plugin \
     connection_url="postgresql://{{username}}:{{password}}@$POSTGRES_URL/postgres?sslmode=disable" \
     allowed_roles=* \
     username="postgres" \
     password="StrongPassword"


Success! Enabled the database secrets engine at: database/
Success! Data written to: database/config/postgresql


## Create a role in Vault

In [8]:
%%bash
cat > ${WORKDIR}/readonly.sql <<EOF
CREATE ROLE "{{name}}" WITH LOGIN PASSWORD '{{password}}' VALID UNTIL '{{expiration}}' INHERIT;
GRANT ro TO "{{name}}";
EOF


In [9]:
%%bash
vault write database/roles/readonly \
      db_name=postgresql \
      creation_statements=@${WORKDIR}/readonly.sql \
      default_ttl=1h \
      max_ttl=24h


Success! Data written to: database/roles/readonly


# Demo

## Request Credentials

In [10]:
%%bash
#via CLI
vault read database/creds/readonly

Key                Value
---                -----
lease_id           database/creds/readonly/7MUFrnllV2OHQFGqt2hd8WZi
lease_duration     1h
lease_renewable    true
password           31EQNt75WZCMmKOVL-vQ
username           v-root-readonly-WGNkZeiRqaPiznNRLjKQ-1684818420


In [11]:
%%bash
#via API
curl -k --header "X-Vault-Token: $VAULT_TOKEN" \
       --silent \
       --request GET \
       $VAULT_ADDR/v1/database/creds/readonly | jq -r .


{
  "request_id": "26caea95-418c-5351-37ee-2ecc8581a7dd",
  "lease_id": "database/creds/readonly/YV8WNcR2Zk2YEChJudelsBQ2",
  "renewable": true,
  "lease_duration": 3600,
  "data": {
    "password": "Hlb1sjmQkb1wiQ-kidaz",
    "username": "v-root-readonly-ee6ldk5Maks22rg4btKR-1684818433"
  },
  "wrap_info": null,
  "warnings": null,
  "auth": null
}


In [12]:
%%bash
## Check leases
vault list sys/leases/lookup/database/creds/readonly

Keys
----
7MUFrnllV2OHQFGqt2hd8WZi
YV8WNcR2Zk2YEChJudelsBQ2


## Rotate Root password

In [ ]:
%%bash
vault write -force database/rotate-root/postgresql

## Clean up